In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install streamlit faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 11.5 MB/s eta 0:00:00


In [3]:
streamlit_code = """
import streamlit as st
import numpy as np
from PIL import Image as PILImage
from keras.preprocessing import image as keras_preprocessing_image
from keras.models import load_model
import json
import sqlite3
from io import BytesIO

# Connect to SQLite database
conn = sqlite3.connect('/content/image_database.db')
cursor = conn.cursor()

# Create table if not exists
cursor.execute('''
    CREATE TABLE IF NOT EXISTS uploaded_images (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        image BLOB
    )
''')
conn.commit()

# Function to preprocess the image
def preprocess_image(img, target_size):
    if img.mode == 'RGBA':
        img = img.convert('RGB')
    img = img.resize(target_size)
    img = keras_preprocessing_image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255.0
    return img

# Load the model
model = load_model("/content/drive/MyDrive/grape_disease_model.h5")

# Load the labels mapping
with open("/content/drive/MyDrive/labels_mapping.json", "r") as json_file:
    labels = json.load(json_file)

# Mapping of links and information for each class
class_info = {
    "Black Rot": {
        "link": "https://www.iffcobazar.in/en/product/satsuma-mancozeb-75-wp#satsuma-1-kg",
        "introduction": "Black Rot is a common grape disease caused by a fungus. It affects the leaves and can lead to significant yield losses. To control Black Rot, consider using chemical solutions like Satsuma Mancozeb 75 WP.",
        "natural_methods": "To prevent Black Rot naturally, maintain good vineyard hygiene. Prune infected parts, provide proper spacing for air circulation, and use organic fungicides. Organic fungicides are always respected in comparison to inorganic compounds."
    },
    "ESCA": {
        "link": "https://www.iffcobazar.in/en/product/moyashi-thiophanate-methyl-70-wp#moyashi-thiophanate-methyl-70-wp-500-gm",
        "introduction": "ESCA, also known as Esca complex, is a grapevine disease caused by various fungi. It affects the wood and can result in wilting and decline. Thiophanate Methyl 70 WP is an effective solution.",
        "natural_methods": "To combat ESCA naturally, ensure proper vineyard sanitation, prune infected parts, and use organic anti-fungal treatments. Additionally, promote overall vine health through balanced nutrition."
    },
    "Leaf Blight": {
        "link": "https://www.iffcobazar.in/en/product/gozaru-copper-oxychloride-50-wp#gozaru-copper-oxychloride-50-wp-500-gm",
        "introduction": "Leaf Blight is a common grape disease caused by bacteria. It affects the leaves, leading to spots and necrosis. Gozaru Copper Oxychloride 50 WP is an effective fungicide.",
        "natural_methods": "For natural control of Leaf Blight, practice proper vineyard management. Remove infected leaves, ensure proper air circulation, and use copper-based organic fungicides. Maintain overall vineyard health."
    }
}

# Streamlit app
st.title("Grape Disease Classification")

# Confidence threshold to filter out predictions with low confidence
confidence_threshold = 0.5

# Section to display stored images
st.header("Stored Images")

# Fetch stored images from the database
stored_images = cursor.execute('SELECT image FROM uploaded_images').fetchall()

# Display stored images
for stored_image in stored_images:
    img_bytes = BytesIO(stored_image[0])
    st.image(img_bytes, caption="Stored Image", use_column_width=True)

# Section for uploading new images
st.header("Upload New Image")

uploaded_file = st.file_uploader("Choose a grape leaf image...", type="jpg")

if uploaded_file is not None:
    # Display the uploaded image
    uploaded_image = PILImage.open(uploaded_file)
    st.image(uploaded_image, caption="Uploaded Image", use_column_width=True)

    # Save the image to the database
    image_bytes = BytesIO()
    uploaded_image.save(image_bytes, format='JPEG')
    cursor.execute('INSERT INTO uploaded_images (image) VALUES (?)', (image_bytes.getvalue(),))
    conn.commit()

    # Make prediction
    processed_image = preprocess_image(uploaded_image, target_size=(224, 224))
    prediction = model.predict(processed_image)
    predicted_class_idx = np.argmax(prediction)
    prediction_probability = np.max(prediction)

    # Check if the prediction probability is above the confidence threshold
    if prediction_probability >= confidence_threshold:
        try:
            predicted_class = list(labels.keys())[predicted_class_idx]
            st.write(f"Prediction: {predicted_class}")
            st.write(f"Probability: {prediction_probability:.2%}")

            # Display links and information based on predicted class
            if predicted_class in class_info:
                st.write(f"Product Link: {class_info[predicted_class]['link']}")
                st.write(f"Introduction: {class_info[predicted_class]['introduction']}")
                st.write(f"Natural Methods: {class_info[predicted_class]['natural_methods']}")
            else:
                st.warning("Additional information not available for this class.")

        except IndexError:
            st.error("Error: Index out of bounds. Check the consistency of labels and model output.")
    else:
        st.write("Unknown")

# Close the database connection
conn.close()



"""


with open('/content/app.py', 'w') as file:
   file.write(streamlit_code)


print("Streamlit app code has been saved to app.py")

Streamlit app code has been saved to app.py


In [4]:
# Download ngrok binary for Linux
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

# Unzip the downloaded file
!unzip ngrok-stable-linux-amd64.zip

# Move ngrok to /usr/local/bin (or any directory in your PATH)
!sudo mv ngrok /usr/local/bin/

# Clean up the downloaded files (optional)
!rm ngrok-stable-linux-amd64.zip

--2024-02-26 20:55:49--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 54.237.133.81, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  19.6MB/s    in 0.7s    

2024-02-26 20:55:50 (19.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [5]:
! curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
  | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
  | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [713 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,622 B]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:9 https://ngrok-agent.s3.amazonaws.com buster/main amd64 Packages [3,730 B]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,070 kB]
Hit:11 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 

In [6]:
!ngrok authtoken 2b2dbgzgUwpEFc6LV0p39tU1zJR_3sMRYo8mQp3KCYbxNYMh



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import subprocess
from multiprocessing import Process

# Function to run a command in a separate process and capture output
def run_command(command):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8') + stderr.decode('utf-8')
    print(f"Command: {command}\nOutput: {output}")
    return output

# Define the commands
streamlit_command = ['streamlit', 'run', '/content/app.py']  # Replace with the actual path to your Streamlit app
ngrok_command = ['ngrok', 'http', '8501']  # Expose Streamlit app on port 8501

# Run each command in a separate process
processes = [
    Process(target=run_command, args=(streamlit_command,)),
    Process(target=run_command, args=(ngrok_command,))
]

# Start all processes
for process in processes:
    process.start()

# Wait for all processes to finish
for process in processes:
    process.join()
